In [1]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performer as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
'''
def solve_cudnn_error():
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
            
solve_cudnn_error()
'''

'\ndef solve_cudnn_error():\n    import tensorflow as tf\n    gpus = tf.config.experimental.list_physical_devices(\'GPU\')\n    if gpus:\n        try:\n            # Currently, memory growth needs to be the same across GPUs\n            for gpu in gpus:\n                tf.config.experimental.set_memory_growth(gpu, True)\n            logical_gpus = tf.config.experimental.list_logical_devices(\'GPU\')\n            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n        except RuntimeError as e:\n            # Memory growth must be set before GPUs have been initialized\n            print(e)\n            \nsolve_cudnn_error()\n'

In [3]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [4]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [5]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [6]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [7]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return tokens

In [8]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [9]:
from matplotlib import pyplot as plt
def plotTrainingLoss(history):
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

In [10]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [11]:
class TestTranslate(unittest.TestCase):
    
    @staticmethod
    def _build_token_dict(token_list):
        token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<BOTM>': 5,
            '<BOT>': 6,
            '<EOT>': 7,
            '<BOM>': 8,
            '<EOM>': 9,
            '<EOTM>': 10,
            '<CR>': 11,
        }
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        print("i am here: " )
        
        max_javaline_length=160  #Max number of lines 
        
        #source_file=[]
        
        '''
        InputFiles = sorted(listdir(Input_Path))
        OutputFiles = sorted(listdir(Location_Output_Path))
        '''
        
        Input_Path = "Trianing/InputTxt"
        Output_Path = "Trianing/InputCSV"
        cases = listdir_fullpath(Input_Path)
        source_max_len = 0
        target_max_len = 0
        token_num = 0
        block_num = 10
        self.sl = 0
        for loop in range(0, round(336/block_num)):
            print("loop: ", loop)
            source_tokens = []
            target_errors=[]
            target_LB=[]
            dirs = block_num if loop < 336//block_num else 336%block_num
            Input_fullpath = []
            Output_fullpath = []
            for i in range(dirs):
                for j in range(1000):
                    in_path = Input_Path + "/" + cases[loop + i] + "/" + str(j)+ "/*.txt"
                    #print("in_path path:", in_path)
                    Input_fullpath += glob.glob(in_path)
            for f in Input_fullpath:
                if isfile(f):
                    source_tokens.append(parseSentence(readcode(f)))
                    print("source_tokens length: ", len(source_tokens))
                #if len(source_tokens)>max_files: break
            out_path = Output_Path + "/" + "test" + str(loop)+".csv"
            Output_fullpath = glob.glob(out_path)
            for f in Output_fullpath:
                if isfile(f):
                    err,lb = readCSV(f)
                    target_errors.append(err)
                    target_LB.append(lb)
                #if len(source_tokens)>max_files: break
            dd=np.asarray(target_errors)
            target_errors=target_errors[0]  
            target_LB=target_LB[0]     
            
            source_tokens2 = []
            target_errors2= []
            target_LB2=[]

            THRESHOLD_FILE_LEN = 64000000
            print("source tokens length:", len(source_tokens))
            for i in range(len(source_tokens)):
                #print (i)
                src = source_tokens[i]
                err = target_errors[i]   
                lb = target_LB[i]                   
                if (len(src)<=THRESHOLD_FILE_LEN):
                    source_tokens2.append(src)
                    target_errors2.append(err)
                    target_LB2.append(lb) 
            print("source_token2 length: ", len(source_tokens2))
            source_tokens = source_tokens2
            target_errors = target_errors2        
            target_LB= target_LB2        

            # Generate dictionaries
            source_token_dict = self._build_token_dict(source_tokens)
            
            # Add special tokens
            encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            
            #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens] 
            
            self.sl = max(list(map(len, encode_tokens))+[self.sl])
            source_max_len = self.sl

            #padding here
            for loop in range(0, round(336/block_num)):
                print("loop:", loop)
                source_tokens = []
                target_errors=[]
                target_LB=[]
                dirs = block_num if loop < 336//block_num else 336%block_num
                Input_fullpath = []
                Output_fullpath = []
                for i in range(dirs):
                    for j in range(1000):
                        in_path = Input_Path + "/" + cases[loop + i] + "/" + str(j) + "/*.txt"
                        Input_fullpath += glob.glob(in_path)
                for f in Input_fullpath:
                    if isfile(f):
                        source_tokens.append(parseSentence(readcode(f)))
                    #if len(source_tokens)>max_files: break
            
            out_path = Output_Path + "/" + "test"+str(loop)+".csv"
            Output_fullpath = glob.glob(out_path)
            for f in Output_fullpath:
                if isfile(f):
                    err, lb = readCSV(f)
                    target_errors.append(err)
                    target_LB.append(lb)
            #if len(source_tokens)>max_files: break
            dd=np.asarray(target_errors)
            print("AAAA: ", dd.shape)
            print("aaaa: " , type(target_errors[0][0]))
            target_errors=target_errors[0]  
            target_LB=target_LB[0]     
            print("XXXX: " , len(source_tokens))
            print("YYYY: " , type(target_errors[0][0]))
            print("ZZZZ: " , len(target_LB))
            print("ZZZZ len(target_LB[0]): " , len(target_LB[0]))

            source_tokens2 = []
            target_errors2= []
            target_LB2=[]

            THRESHOLD_FILE_LEN = 64000000
            print("XXXX1: " , len(source_tokens))
            for i in range(len(source_tokens)): 
                #print (i)
                src = source_tokens[i]
                err = target_errors[i]   
                lb = target_LB[i]                   
                if (len(src)<=THRESHOLD_FILE_LEN):
                    source_tokens2.append(src)
                    target_errors2.append(err)
                    target_LB2.append(lb) 
            source_tokens = source_tokens2
            target_errors= target_errors2        
            target_LB = target_LB2        

            print("XXXX2: " , len(source_tokens))
            
            
            # Generate dictionaries
            source_token_dict = self._build_token_dict(source_tokens)
            
            
            
            # Add special tokens
            encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            print("encode_tokens1: ", encode_tokens)
            encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
            print("encode_tokens2: ", encode_tokens)
            encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
            print("encode_input1: ", encode_input)
            token_num =len(source_token_dict)
            print("token num: ", token_num)
            print(token_num)
            print(type(token_num))
        
        
            print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)

            
            #x=[np.array(encode_input * 1)]
            #y=[np.array(target_errors * 1),np.array(target_LB * 1)]
        
            #print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)
              
            ####  Split the data set into train and test_model
            x=np.array(encode_input)
            y=list(zip(np.array(target_errors), np.array(target_LB)))

            print("x.shape: ", x.shape)
            print("y length: ", len(y))
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

            y_train = list(zip(*y_train))  
            y_train[0] = np.array(y_train[0])
            y_train[1] = np.array(y_train[1])

            y_train[1] = to_categorical(y_train[1], num_classes=max_javaline_length) #將類別向量轉換為二進制矩陣
            #y_train = list(zip(y_train[0], y_train[1]))

            print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX:" , y_train[0].shape)
            print("y_train[1].shape", y_train[1].shape)
            print("???????????: y_train.length ", len(y_train))
            print("???????????: y_train.[1] type ", type(y_train[1]))
            y_train[1] = np.split(y_train[1], indices_or_sections=len(target_LB[0]), axis=1)
            y_train[1] = [  np.squeeze(elm, axis=1) for elm in y_train[1] ]
            print("after change->len(y_train[1].shape)", len(y_train[1]) )
            
            y_test = list(zip(*y_test))
            y_test[0] = np.array(y_test[0])
            y_test[1] = np.array(y_test[1])
            y_test[1] = to_categorical(y_test[1], num_classes=max_javaline_length) 
            #y_test = list(zip(y_test[0], y_test[1])) 
            y_test[1] = np.split(y_test[1], indices_or_sections=len(target_LB[0]), axis=1) 
            y_test[1] = [  np.squeeze(elm, axis=1) for elm in y_test[1] ]           
              
            saveTestTrainData("test_models/x_test.npy", x_test)
            saveTestTrainData("test_models/y_test0.npy", y_test[0])
            saveTestTrainData("test_models/y_test1.npy", y_test[1])
            saveTestTrainData("train_models/x_train.npy", x_train)  
            saveTestTrainData("train_models/y_train0.npy", y_train[0])#target
            saveTestTrainData("train_models/y_train1.npy", y_train[1])#lb
            
            ####
            #if you only need to train model don't use this
            print("y_train[0].shape",y_train[0].shape)
            print("len(y_train[1])",len(y_train[1]))
            print("y_train[1][0].shape",y_train[1][0].shape)
        
            '''
            #Build & fit model
            #Set model para    
            model = tfr.get_model(
                max_input_len= (source_max_len),
                max_javaline_length=max_javaline_length,
                errNum=36,
                lbNum=len(target_LB[0]),
                token_num=len(source_token_dict),
                embed_dim=32, #32,
                encoder_num=4, #2 max = 6
                head_num=4,#4
                hidden_dim=64, #128
                dropout_rate=0.05 #0.05
            )
            #'LNout0'
        
            losses = {    
                "error_feed_forward_output1": "binary_crossentropy"}
                #"error_feed_forward_output2": "categorical_crossentropy",            
        
        
            lossWeights = {"error_feed_forward_output1": 1.0  }#"error_feed_forward_output2": 1.0}
        
            for i in range(len(target_LB[0])):  #列出 len(target_LB[0]) 組網路層
                name = "LNout"+str(i)
                losses[name] = "categorical_crossentropy"
                lossWeights[name] = 1 #error_feed_forward_output2[] weight

            
            model.compile(optimizer=Adam(learning_rate=0.001), loss=losses, loss_weights=lossWeights)
        
            history=model.fit(
                x=x_train,
                y= [y_train[0]] + y_train[1],
                epochs=1, #100 200 500
                batch_size=32,
            )
            plotTrainingLoss(history)
            model.save("test_model1.h5")
            saveDictionary(source_token_dict, "source_token_dict.pickle")
        
            model, source_token_dict = load("test_model1.h5")

            # Predict
        
            x_train_loaded = loadTestTrainData("train_models/x_train_500.npy") 
            y_train_loaded_0 =   loadTestTrainData("train_models/y_train0_500.npy")
            y_train_loaded_1 =   loadTestTrainData("train_models/y_train1_500.npy") 

            x_test_loaded =  loadTestTrainData("test_models/x_test_500.npy") 
            y_test_loaded_0 =   loadTestTrainData("test_models/y_test0_500.npy")
            y_test_loaded_1 =   loadTestTrainData("test_models/y_test1_500.npy") 
        
        
            out1, out2 = tfr.decode(
                model,
                #encode_input,
                x_test_loaded,max_len=source_max_len
            )
            print("out1: ", out1)
            #print("target_errors: ",target_errors) 
            print("target_errors: ", y_test_loaded_0)        
            print("out2: ", out2)
            #print("target_LB: ", target_LB)
            print("target_LB: ", y_test_loaded_1)
        
            #model.summary()
        
            print("type(y_train_loaded_0)",type(y_train_loaded_0))
            print("type(y_train_loaded_1)",type(y_train_loaded_1))
            print("y_train_loaded[0].shape",y_train_loaded_0.shape)

            print("len(y_train_loaded[1])",len(y_train_loaded_1))
            print("y_train_loaded[1][0].shape",y_train_loaded_1[0].shape)
            train_loss = model.evaluate(x_train_loaded, [y_train_loaded_0] + list(y_train_loaded_1) )
            print("train_loss",train_loss)
        
            test_loss = model.evaluate(x_test_loaded, [y_test_loaded_0] + list(y_test_loaded_1) )
            print("test_loss",test_loss)
            '''
    def getsource_max_lan(self):
        return self.sl

In [12]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [13]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [14]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [15]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [16]:
def load(model_name):
        import sys
        #sys.path.append('/content/drive/MyDrive/Final_Edition_include_model')
        sys.path.append("Performer\Performer_local_V3\keras_layer_normalization")
        sys.path.append("Performer\Performer_local_V3\keras_performer")
        sys.path.append("Performer\Performer_local_V3\keras_position_wise_feed_forward")
        sys.path.append("Performer\Performer_local_V3\tensorflow_fast_attention")

        from keras_performer import performer
        from tensorflow import keras
        from keras_embed_sim import EmbeddingRet, EmbeddingSim
        from keras_pos_embd import TrigPosEmbedding
        from tensorflow_fast_attention.fast_attention import  Attention, SelfAttention
        from keras_position_wise_feed_forward.feed_forward import FeedForward  

        co = performer.get_custom_objects()

        model = keras.models.load_model(model_name, custom_objects= co)
        source_token_dict = loadDictionary("source_token_dict.pickle")
       # t = loadDictionary(target_token_dict, 'target_token_dict.pickle')
       # t_inv = loadDictionary(target_token_dict_inv, 'target_token_dict_inv.pickle')
        return model, source_token_dict,

In [17]:
x=TestTranslate()
x.test_translate()

i am here: 
loop:  0
source_tokens length:  1
source_tokens length:  2
source_tokens length:  3
source_tokens length:  4
source_tokens length:  5
source_tokens length:  6
source_tokens length:  7
source_tokens length:  8
source_tokens length:  9
source_tokens length:  10
source_tokens length:  11
source_tokens length:  12
source_tokens length:  13
source_tokens length:  14
source_tokens length:  15
source_tokens length:  16
source_tokens length:  17
source_tokens length:  18
source_tokens length:  19
source_tokens length:  20
source_tokens length:  21
source_tokens length:  22
source_tokens length:  23
source_tokens length:  24
source_tokens length:  25
source_tokens length:  26
source_tokens length:  27
source_tokens length:  28
source_tokens length:  29
source_tokens length:  30
source_tokens length:  31
source_tokens length:  32
source_tokens length:  33
source_tokens length:  34
source_tokens length:  35
source_tokens length:  36
source_tokens length:  37
source_tokens length:  38


source_tokens length:  400
source_tokens length:  401
source_tokens length:  402
source_tokens length:  403
source_tokens length:  404
source_tokens length:  405
source_tokens length:  406
source_tokens length:  407
source_tokens length:  408
source_tokens length:  409
source_tokens length:  410
source_tokens length:  411
source_tokens length:  412
source_tokens length:  413
source_tokens length:  414
source_tokens length:  415
source_tokens length:  416
source_tokens length:  417
source_tokens length:  418
source_tokens length:  419
source_tokens length:  420
source_tokens length:  421
source_tokens length:  422
source_tokens length:  423
source_tokens length:  424
source_tokens length:  425
source_tokens length:  426
source_tokens length:  427
source_tokens length:  428
source_tokens length:  429
source_tokens length:  430
source_tokens length:  431
source_tokens length:  432
source_tokens length:  433
source_tokens length:  434
source_tokens length:  435
source_tokens length:  436
s

source_tokens length:  820
source_tokens length:  821
source_tokens length:  822
source_tokens length:  823
source_tokens length:  824
source_tokens length:  825
source_tokens length:  826
source_tokens length:  827
source_tokens length:  828
source_tokens length:  829
source_tokens length:  830
source_tokens length:  831
source_tokens length:  832
source_tokens length:  833
source_tokens length:  834
source_tokens length:  835
source_tokens length:  836
source_tokens length:  837
source_tokens length:  838
source_tokens length:  839
source_tokens length:  840
source_tokens length:  841
source_tokens length:  842
source_tokens length:  843
source_tokens length:  844
source_tokens length:  845
source_tokens length:  846
source_tokens length:  847
source_tokens length:  848
source_tokens length:  849
source_tokens length:  850
source_tokens length:  851
source_tokens length:  852
source_tokens length:  853
source_tokens length:  854
source_tokens length:  855
source_tokens length:  856
s

source_tokens length:  1243
source_tokens length:  1244
source_tokens length:  1245
source_tokens length:  1246
source_tokens length:  1247
source_tokens length:  1248
source_tokens length:  1249
source_tokens length:  1250
source_tokens length:  1251
source_tokens length:  1252
source_tokens length:  1253
source_tokens length:  1254
source_tokens length:  1255
source_tokens length:  1256
source_tokens length:  1257
source_tokens length:  1258
source_tokens length:  1259
source_tokens length:  1260
source_tokens length:  1261
source_tokens length:  1262
source_tokens length:  1263
source_tokens length:  1264
source_tokens length:  1265
source_tokens length:  1266
source_tokens length:  1267
source_tokens length:  1268
source_tokens length:  1269
source_tokens length:  1270
source_tokens length:  1271
source_tokens length:  1272
source_tokens length:  1273
source_tokens length:  1274
source_tokens length:  1275
source_tokens length:  1276
source_tokens length:  1277
source_tokens length

source_tokens length:  1635
source_tokens length:  1636
source_tokens length:  1637
source_tokens length:  1638
source_tokens length:  1639
source_tokens length:  1640
source_tokens length:  1641
source_tokens length:  1642
source_tokens length:  1643
source_tokens length:  1644
source_tokens length:  1645
source_tokens length:  1646
source_tokens length:  1647
source_tokens length:  1648
source_tokens length:  1649
source_tokens length:  1650
source_tokens length:  1651
source_tokens length:  1652
source_tokens length:  1653
source_tokens length:  1654
source_tokens length:  1655
source_tokens length:  1656
source_tokens length:  1657
source_tokens length:  1658
source_tokens length:  1659
source_tokens length:  1660
source_tokens length:  1661
source_tokens length:  1662
source_tokens length:  1663
source_tokens length:  1664
source_tokens length:  1665
source_tokens length:  1666
source_tokens length:  1667
source_tokens length:  1668
source_tokens length:  1669
source_tokens length

...

source_tokens length:  8902
source_tokens length:  8903
source_tokens length:  8904
source_tokens length:  8905
source_tokens length:  8906
source_tokens length:  8907
source_tokens length:  8908
source_tokens length:  8909
source_tokens length:  8910
source_tokens length:  8911
source_tokens length:  8912
source_tokens length:  8913
source_tokens length:  8914
source_tokens length:  8915
source_tokens length:  8916
source_tokens length:  8917
source_tokens length:  8918
source_tokens length:  8919
source_tokens length:  8920
source_tokens length:  8921
source_tokens length:  8922
source_tokens length:  8923
source_tokens length:  8924
source_tokens length:  8925
source_tokens length:  8926
source_tokens length:  8927
source_tokens length:  8928
source_tokens length:  8929
source_tokens length:  8930
source_tokens length:  8931
source_tokens length:  8932
source_tokens length:  8933
source_tokens length:  8934
source_tokens length:  8935
source_tokens length:  8936
source_tokens length

source_tokens length:  9262
source_tokens length:  9263
source_tokens length:  9264
source_tokens length:  9265
source_tokens length:  9266
source_tokens length:  9267
source_tokens length:  9268
source_tokens length:  9269
source_tokens length:  9270
source_tokens length:  9271
source_tokens length:  9272
source_tokens length:  9273
source_tokens length:  9274
source_tokens length:  9275
source_tokens length:  9276
source_tokens length:  9277
source_tokens length:  9278
source_tokens length:  9279
source_tokens length:  9280
source_tokens length:  9281
source_tokens length:  9282
source_tokens length:  9283
source_tokens length:  9284
source_tokens length:  9285
source_tokens length:  9286
source_tokens length:  9287
source_tokens length:  9288
source_tokens length:  9289
source_tokens length:  9290
source_tokens length:  9291
source_tokens length:  9292
source_tokens length:  9293
source_tokens length:  9294
source_tokens length:  9295
source_tokens length:  9296
source_tokens length

source_tokens length:  9577
source_tokens length:  9578
source_tokens length:  9579
source_tokens length:  9580
source_tokens length:  9581
source_tokens length:  9582
source_tokens length:  9583
source_tokens length:  9584
source_tokens length:  9585
source_tokens length:  9586
source_tokens length:  9587
source_tokens length:  9588
source_tokens length:  9589
source_tokens length:  9590
source_tokens length:  9591
source_tokens length:  9592
source_tokens length:  9593
source_tokens length:  9594
source_tokens length:  9595
source_tokens length:  9596
source_tokens length:  9597
source_tokens length:  9598
source_tokens length:  9599
source_tokens length:  9600
source_tokens length:  9601
source_tokens length:  9602
source_tokens length:  9603
source_tokens length:  9604
source_tokens length:  9605
source_tokens length:  9606
source_tokens length:  9607
source_tokens length:  9608
source_tokens length:  9609
source_tokens length:  9610
source_tokens length:  9611
source_tokens length

source_tokens length:  9917
source_tokens length:  9918
source_tokens length:  9919
source_tokens length:  9920
source_tokens length:  9921
source_tokens length:  9922
source_tokens length:  9923
source_tokens length:  9924
source_tokens length:  9925
source_tokens length:  9926
source_tokens length:  9927
source_tokens length:  9928
source_tokens length:  9929
source_tokens length:  9930
source_tokens length:  9931
source_tokens length:  9932
source_tokens length:  9933
source_tokens length:  9934
source_tokens length:  9935
source_tokens length:  9936
source_tokens length:  9937
source_tokens length:  9938
source_tokens length:  9939
source_tokens length:  9940
source_tokens length:  9941
source_tokens length:  9942
source_tokens length:  9943
source_tokens length:  9944
source_tokens length:  9945
source_tokens length:  9946
source_tokens length:  9947
source_tokens length:  9948
source_tokens length:  9949
source_tokens length:  9950
source_tokens length:  9951
source_tokens length

IndexError: list index out of range

In [ ]:
#set model para
model_name = "test_model1.h5"
x_test_model = "x_test_500.npy" 
y_test_mdodel1 = "y_test0_500.npy" 
y_test_mdodel2 = "y_test1_500.npy" 

In [ ]:
#load model
def loadmodel(model_name, x_test_model, y_test_mdodel1, y_test_mdodel2):
    import numpy as np
    #load model and dic ps. dic is not use
    model, source_token_dict = load(model_name)
    #print(model.summary())
    #load
    
    '''
    Para:
        x_test_loaded  : answer model        (include type and lineblock)
        y_test_loaded_0: predict model       (type)
        y_test_loaded_1: predict model       (lineblock)
        out1           : answer model output (type) 
        out2           : answer model output (lineblock)
    '''
    
    x_test_loaded = loadTestTrainData("test_models/" + x_test_model) 
    y_test_loaded_0 = loadTestTrainData("test_models/" + y_test_mdodel1)
    y_test_loaded_1 = loadTestTrainData("test_models/" + y_test_mdodel2) 
    
    ''' <-------dust switch
    print("y_test_loaded_1 shape: ", y_test_loaded_1.shape)
    print("x_test_loaded length: ", len(x_test_loaded))
    #'''
    
    out1, out2 = tfr.decode(model,x_test_loaded, max_len = x.getsource_max_lan())
    
    #==============show org result================
    ''' <-------dust switch
    print("y_test_loaded_0 shape: ", y_test_loaded_0.shape)
    print("y_test_loaded_1 shape: ", y_test_loaded_1.shape)
    print("y_test_loaded_0[0] result:", (y_test_loaded_0[0])) #Error_type #vs out1
    print("y_test_loaded_1[0][1] result:", (y_test_loaded_1[0][1])) #Line_Block #vs out2
    print("out1 shape: ", (out1).shape)#prob upper then 0.5
    print("out2[0] shape: ", (out2[0]).shape)#prob upper then 0.5
    print("out2 length: ", len(out2))
    print("out2[0] length: ", len(out2[0]))#prob lb
    #'''
    
    #=============================================
    test_ep = np.around(out1)
    test_lb = np.around(out2)
    ans_ep = np.around(y_test_loaded_0)
    ans_lb = np.around(y_test_loaded_1)
    #==============show toint result==============
    
    ''' <-------dust switch
    print(test_ep[1])
    print(test_lb[1])
    print(ans_ep[1])
    print([ans_lb[1]])
    #'''
    #=============================================
    return test_ep, ans_ep, test_lb, ans_lb

In [ ]:
def intersect(pre_errortype, ans_errortype):
    #print("inter :", pre_errortype)
    #print("inter: ", ans_errortype)
    #ans_errortype = float(ans_errortype)
    #ref = "https://www.796t.com/post/Mjc4am8=.html"
    inter = [pre_value for pre_value, ans_value in zip(pre_errortype, ans_errortype) if (pre_value == ans_value == 1)]
    inter_two = [pre_value for pre_value, ans_value in zip(pre_errortype, ans_errortype) if (pre_value == ans_value)]
    #print("sort inter: ", inter)
    return inter, inter_two

In [ ]:
#get new ans array contains only 1 in array 
def ans_typefilter(ans_errortype):
    #print("org ans type: ", ans_errortype)
    new_ans_errortype = [value for value in ans_errortype if value == 1]
    #print("new ans errortype: ", new_ans_errortype)
    return new_ans_errortype

In [ ]:
#get new pre array contains only 1 in array 
def pre_typefilter(pre_errortype):
    #print("org error type: ", pre_errortype)
    new_pre_errortype = [value for value in pre_errortype if value == 1]
    #print("new pre errortype: ", new_pre_errortype)
    return new_pre_errortype

In [ ]:
#calculate error type score
#Note:
    #Predict score:
        # score = inter of ans and pre divide by len of pre
    #Recall score:
        # score = inter of ans and pre divide by len of ans
    #Accuracy score:
        # score = inter_two of ans and pre divide by original len of pre
def errortype_score(pre_errortype, ans_errortype):
    #find pre length
    new_pre_errortype = pre_typefilter(pre_errortype) #make new array for score
    pre_length = len(new_pre_errortype) #get lenght from ans_type
    #find ans length
    new_ans_errortype = ans_typefilter(ans_errortype) #make new array for score
    ans_length = len(new_ans_errortype) #get lenght from ans_type
    #print("ans length: ", ans_length)
    inter, inter_two = intersect(pre_errortype, ans_errortype) #get intersection
    inter_length = len(inter)
    inter_two_length = len(inter_two)
    #print("inter_length: ", inter_length)
    #print("pre length: ", pre_length) 
    
    #calculate Predict score 
    if (inter_length == 0 and pre_length == 0):
        pre_score = 1
    elif (pre_length == 0):
        pre_score = 0
    else:
        pre_score = inter_length/pre_length 
    #print("predict score: ", pre_score) #show pre score
    
    #calculate Recall score
    if (inter_length == 0 and ans_length == 0):
        rec_score = 1
    elif (ans_length == 0):
        rec_score = 0
    else:
        rec_score = inter_length/ans_length
    #print("recall score: ", rec_score) #show ans score
    
    #calculate Accuarcy score
    acc_score = inter_two_length/len(pre_errortype)
    
    return pre_score, rec_score , acc_score#return float

In [ ]:
#show total perdict score and recall score
def errortype_totalscore(pre_errortype,ans_errortype):
    #initial para
    pre_total = 0.0
    rec_total = 0.0
    acc_total = 0.0
    #get each score then get total and avg score
    for i in range(len(pre_errortype)):
        #print("Sample: ", i)
        pre, rec, acc = errortype_score(pre_errortype[i], ans_errortype[i])
        pre_total = pre_total + pre
        rec_total = rec_total + rec
        acc_total = acc_total + acc
    print("pre_total: ", pre_total)
    print("rec_total: ", rec_total)
    print("acc_total: ", acc_total)
    pre_avg_score = pre_total/len(pre_errortype)
    rec_avg_socre = rec_total/len(ans_errortype)
    acc_avg_score = acc_total/len(ans_errortype)
    print("avg_pre: ", pre_avg_score)#pre_total/len(pre_errortype))
    print("avg_rec: ", rec_avg_socre)#rec_total/len(ans_errortype))
    print("avg_acc: ", acc_avg_score)#acc_total/len(ans_errortype))
    return pre_avg_score, rec_avg_socre, acc_avg_score

In [ ]:
def get_start_end_line(pre_begin_index, pre_end_index, ans_begin_index, ans_end_index):
    #Note:
            #use softmax: [0.....150] total == 1
            #begin: index[0]
            #end: index[0]
        
    #get [value] first and get [value] again to get value
    pre_begin_line = pre_begin_index[0]
    pre_begin_line = pre_begin_line[0]
    pre_end_line = pre_end_index[0]
    pre_end_line = pre_end_line[0]
    ans_begin_line = ans_begin_index[0]
    ans_begin_line = ans_begin_line[0]
    ans_end_line = ans_end_index[0]
    ans_end_line = ans_end_line[0]
    
    return pre_begin_line, pre_end_line, ans_begin_line, ans_end_line

In [ ]:
def line_adjustment(line):
    if(line > 1):
        line = line-1
        return line
    else:
        return line

In [ ]:
# get each pre and ans lineblocks and make them to pre begin/end ans begin/end array
def make_lineblock(begin, end):
    if (end < begin):
        block = [value for value in range(end, begin+1)]
        return block
    else:
        block = [value for value in range(begin, end+1)]
        return block

In [ ]:
# ref = "https://www.geeksforgeeks.org/python-intersection-two-lists/"
def lineblock_intersect(pre_block, ans_block):
    inter = [value for value in pre_block if value in ans_block]
    return inter

In [ ]:
# return index with int datatype 
def get_block_index(pre_begins, pre_ends, ans_begins, ans_ends):
    pre_begin = int(pre_begins)
    pre_end = int(pre_ends)
    ans_begin = int(ans_begins)
    ans_end = int(ans_ends)
    #print(pre_begin, pre_end, ans_begin, ans_end)
    return pre_begin, pre_end, ans_begin, ans_end

In [ ]:
#get new pre array contains only 1 in array 
def linefilter(line_block):
    #print("org error type: ", pre_errortype)
    new_line = [value for value in line_block if value == 1]
    #print("new pre errortype: ", new_pre_errortype)
    return new_line

In [ ]:
 #Note:
    #Predict score:
        # score = inter of ans and pre divide by len of pre
    #Recall score:
        # score = inter of ans and pre divide by len of ans
    #                 b   s   l
    #line sturcture [84][36][150]
def errorline_totalscore(pre_errorline, ans_errorline):
    #get sample size
    sample_size = len(pre_errorline[1])
    #get block size
    lineblock_size = len(pre_errorline)
    total_pre = 0.0
    total_rec = 0.0
    total_sample_pre = 0.0
    total_sample_rec = 0.0
    for sample in range(sample_size):
        sample_totalline_pre = 0.0
        sample_totalline_rec = 0.0
        for lineblock in range(0, lineblock_size, 2):
            #give index blocks value
            pre_begin = pre_errorline[lineblock][sample]
            pre_end = pre_errorline[lineblock+1][sample]
            ans_begin = ans_errorline[lineblock][sample]
            ans_end = ans_errorline[lineblock+1][sample]
            ''' <-------dust switch
            #show error block value
            print("pre_errorline begin[lineblock][sample]: ", lineblock, " ", sample)
            print(pre_begin)
            print("pre_errorline begin[lineblock][sample]: ", lineblock+1, " ", sample)
            print(pre_end)
            print("ans_errorline begin [lineblock][sample]: ", lineblock, " ", sample)
            print(ans_begin)
            print("ans_errorline end[lineblock][sample]: ", lineblock+1, " ", sample)
            print(ans_end)
            #'''
            #get block start/end index
            pre_begin_index = np.where(pre_begin == 1)
            pre_end_index = np.where(pre_end == 1)
            ans_begin_index = np.where(ans_begin == 1)
            ans_end_index = np.where(ans_end == 1)
            ''' <-------dust switch
            #show start/end index
            print("pre_begin_index: ", pre_begin_index)
            print("pre_begin_index type: ", type(pre_begin_index))
            print("pre_end_index: ", pre_end_index)
            print("ans_begin_index: ", ans_begin_index)
            print("ans_end_index: ", ans_end_index)
            #'''
            #give start value
            #Note:
                #use softmax: [0.....150] total == 1
                #begin: index[0]
                #end: index[0]

            #get [value] first and get [value] again to get value
            pre_begin_line, pre_end_line, ans_begin_line, ans_end_line = get_start_end_line(
                                                                                            pre_begin_index, 
                                                                                            pre_end_index, 
                                                                                            ans_begin_index, 
                                                                                            ans_end_index
                                                                                            )
            ''' <-------dust switch
            #show start/end index
            print("pre_begin_line: ", pre_begin_line)
            print("pre_end_line: ", pre_end_line)
            print("ans_begin_line: ", ans_begin_line)
            print("ans_end_line: ", ans_end_line)
            #'''
            ''' <-------dust switch
            #'''
            #make line downgrade for 1
            #Note:
            #before:  0 -1  1  2  3  4
            #after:   0  1 <-----abs  
                        #1  2  3  4
                    #[0, 1, 2, 3, 4, 5]
            pre_begin_line = line_adjustment(pre_begin_line)
            pre_end_line = line_adjustment(pre_end_line)
            ans_begin_line = line_adjustment(ans_begin_line)
            ans_end_line = line_adjustment(ans_end_line)
            ''' <-------dust switch
            print("pre_begin_line: ", pre_begin_line)
            print("pre_end_line: ", pre_end_line)
            print("ans_begin_line: ",ans_begin_line)
            print("ans_end_line: ", ans_end_line)
            #'''
            pre_lineblock = make_lineblock(pre_begin_line, pre_end_line)
            ans_lineblock = make_lineblock(ans_begin_line, ans_end_line)
            #print("pre_lineblock: ", pre_lineblock)
            #print("ans_lineblock: ", ans_lineblock)
            inter = lineblock_intersect(pre_lineblock, ans_lineblock)
            #print("inter: ", inter)
            inter_length = len(inter)
            pre_length = len(pre_lineblock)
            ans_length = len(ans_lineblock)
            #print("inter length: ", inter_length)
            #print("pre_length: ", pre_length)
            #print("ans_length: ", ans_length)
            #count pre_score
            if (inter_length == 0 and pre_length == 0):
                pre_score = 1
            elif (pre_length == 0):
                pre_score = 0
            else :
                pre_score = inter_length/pre_length
            #count rec_score
            if (inter_length == 0 and ans_length == 0):
                rec_score = 1  
                #
            elif (ans_length == 0):
                rec_score = 0
            else:
                rec_score = inter_length/ans_length
            #print("Sample pre_score: ", pre_score)
            #print("Sample rec_score: ", rec_score)
            #cal each sample lineblock score
            sample_totalline_pre += pre_score
            sample_totalline_rec += rec_score
            #print("Sample total_pre: ", sample_pre)
            #print("Sample total_rec: ", sample_rec)
        #cal total sample score
        total_sample_pre += sample_totalline_pre/len(lineblock_size/2)
        total_sample_rec += sample_totalline_rec/len(lineblock_size/2)
    avg_pre = total_sample_pre/sample_size
    avg_rec = total_sample_rec/sample_size
    return avg_pre, avg_rec

In [ ]:
def error_type_F_score(pre_score, rec_score):
    f_one = (2*pre_score*rec_score)/(pre_score + rec_score)
    f_two = (3*pre_score*rec_score)/((2*pre_score) + rec_score)
    f_pointfive = (3*pre_score*rec_score)/(pre_score + (2*rec_score))
    print("F_one: ", f_one)
    print("F_two: ", f_two)
    print("F_pointfive: ", f_pointfive)
    

In [ ]:
def error_line_F_score(pre_score, rec_score):
    f_one = (2*pre_score*rec_score)/(pre_score + rec_score)
    f_two = (3*pre_score*rec_score)/((2*pre_score) + rec_score)
    f_pointfive = (3*pre_score*rec_score)/(pre_score + (2*rec_score))
    print("F_one: ", f_one)
    print("F_two: ", f_two)
    print("F_pointfive: ", f_pointfive)


In [ ]:
pre_errortype, ans_errortype, pre_errorline, ans_errorline = loadmodel(model_name, x_test_model, y_test_mdodel1, y_test_mdodel2)
#print(len(pre_errortype))

'''
work : 
    fix 3 dim prob: 
        out2 only show two dim
        score need three dim: out2: 2D, y_test_loaded_1: 3D
    issue:
        y_test_loaded_1: dim is not expected...
    ==================================================================================
    expect :
    y_test_loaded_0 : [sample = 36][error type = 36] correct v
    out1 :            [sample = 36][error type = 36] correct v
    y_test_loaded_1 : [sample = 36][block = 84][begin and end :line total = 150]
    out2 :            [sample = 36][block = 84][begin and end :line total = 150]
    ==================================================================================
    real : 
            out2 shape            = [36][150]    ------>dim should be [36][84][150]
            y_test_loaded_1 shape = [84][36][150]------>dim should be [36][84][150]
    ==================================================================================
    ref in line work space
'''


In [ ]:
#check all sample error type score
avg_pre, avg_rec, avg_acc = errortype_totalscore(pre_errortype, ans_errortype)
error_type_F_score(avg_pre, avg_rec)

In [ ]:
#check all sample error line score
avg_pre, avg_rec = errorline_totalscore(pre_errorline, ans_errorline)
error_line_F_score(avg_pre, avg_rec)

In [ ]:
#work space...


import numpy as np 
#len(ans_errortype)
sample_size = len(pre_errorline[1])
lineblock_size = len(pre_errorline)
'''
Note:
    pre_errorline[lineblock][sample]
    ans_errorline[lineblock][sample]
Para:
'''


#''' <-------dust switch
print("sample size: ", sample_size)
print("lineblock size: ", lineblock_size)
#'''
total_pre_score = 0.0
total_rec_score = 0.0
for sample in range(sample_size):
    for lineblock in range(0, lineblock_size, 2):
        #give index blocks value
        pre_begin = pre_errorline[lineblock][sample]
        pre_end = pre_errorline[lineblock+1][sample]
        ans_begin = ans_errorline[lineblock][sample]
        ans_end = ans_errorline[lineblock+1][sample]
        
        ''' <-------dust switch
        #show error block value
        print("pre_errorline begin[lineblock][sample]: ", lineblock, " ", sample)
        print(pre_begin)
        print("pre_errorline begin[lineblock][sample]: ", lineblock+1, " ", sample)
        print(pre_end)
        print("ans_errorline begin [lineblock][sample]: ", lineblock, " ", sample)
        print(ans_begin)
        print("ans_errorline end[lineblock][sample]: ", lineblock+1, " ", sample)
        print(ans_end)
        #'''
        #get block start/end index
        pre_begin_index = np.where(pre_begin == 1)
        pre_end_index = np.where(pre_end == 1)
        ans_begin_index = np.where(ans_begin == 1)
        ans_end_index = np.where(ans_end == 1)
        
        ''' <-------dust switch
        #show start/end index
        print("pre_begin_index: ", pre_begin_index)
        print("pre_begin_index type: ", type(pre_begin_index))
        print("pre_end_index: ", pre_end_index)
        print("ans_begin_index: ", ans_begin_index)
        print("ans_end_index: ", ans_end_index)
        #'''
        
        #give start value
        #Note:
            #use softmax: [0.....150] total == 1
            #begin: index[0]
            #end: index[0]
        
        #get [value] first and get [value] again to get value
        #pre_begin_line, pre_end_line, ans_begin_line, ans_end_line = get_start_end_line(pre_begin_index, pre_end_index, ans_begin_index, ans_end_index)
        
        pre_begin_line = pre_begin_index[0]
        #pre_begin_line = pre_begin_line[0]
        pre_end_line = pre_end_index[0]
        #pre_end_line = pre_end_line[0]
        ans_begin_line = ans_begin_index[0]
        ans_begin_line = ans_begin_line[0]
        ans_end_line = ans_end_index[0]
        ans_end_line = ans_end_line[0]
        
        #''' <-------dust switch
        #show start/end index
        print("pre_begin_line: ", pre_begin_line)
        print("pre_end_line: ", pre_end_line)
        print("ans_begin_line: ", ans_begin_line)
        print("ans_end_line: ", ans_end_line)
        #'''
        
        
        
        #make intersection
        
        ''' <-------dust switch
        print("inter_begin: ", inter_begin)
        print("inter_all_begin: ", inter_all_begin)
        #'''
        
        
'''
final decide: find index and build ----> [1 2 3 4 5 6] vs [2 3 4 5 6 7] to make intercestion
'''

In [ ]:
#Note:
    #Predict score:
        # score = inter of ans and pre divide by len of pre
    #Recall score:
        # score = inter of ans and pre divide by len of ans
    #Accuracy score:
        # score = inter_two of ans and pre divide by original len of pre

#case 1:  n  n vs  n  n 
#case 2:  n  n vs -1 -1
#case 3: -1 -1 vs -1 -1
#case 4: -1  n vs  n  n
#case 5: -1  n vs -1  n
pre_begin_line = 5
pre_end_line = 7
ans_begin_line = 3
ans_end_line = 2
pre_begin_line = line_adjustment(pre_begin_line)
pre_end_line = line_adjustment(pre_end_line)
ans_begin_line = line_adjustment(ans_begin_line)
ans_end_line = line_adjustment(ans_end_line)

print("pre_begin_line: ", pre_begin_line)
print("pre_end_line: ", pre_end_line)
print("ans_begin_line: ",ans_begin_line)
print("ans_end_line: ", ans_end_line)
pre_lineblock = make_lineblock(pre_begin_line, pre_end_line)
ans_lineblock = make_lineblock(ans_begin_line, ans_end_line)
print("pre_lineblock: ", pre_lineblock)
print("ans_lineblock: ", ans_lineblock)
inter = lineblock_intersect(pre_lineblock, ans_lineblock)
print("inter: ", inter)
inter_length = len(inter)
pre_length = len(pre_lineblock)
ans_length = len(ans_lineblock)
print("inter length: ", inter_length)
print("pre_length: ", pre_length)
print("ans_length: ", ans_length)
if (inter_length == 0 and pre_length == 0):
    pre_score = 1
    # issue 7.12ver 
elif (pre_length == 0):
    pre_score = 0
else :
    pre_score = inter_length/pre_length
#    
if (inter_length == 0 and ans_length == 0):
    rec_score = 1  
    #
elif (ans_length == 0):
    rec_score = 0
else:
    rec_score = inter_length/ans_length
#print("Sample pre_score: ", pre_score)
total_pre += pre_score
#print("Sample rec_score: ", rec_score)
total_rec += rec_score
#print("Sample total_pre: ", total_pre)
#print("Sample total_rec: ", total_rec)
avg_pre = total_pre/len(pre_begins)
avg_rec = total_rec/len(ans_begins)
